In [3]:
import csv
import numpy as np
import pandas as pd
import string

%matplotlib inline

# Functions

In [4]:
def format_table_output(raw, title = None):
    pre_text = '\\begin{table}\n\\small\n' 
    
    if title:
        pre_text = pre_text + '\\caption*{' + title + '}\n'
    
    post_text = '\\end{table}'
    return pre_text + raw + post_text

# Load Data

In [5]:
# Processed disclosure data
data_605 = pd.read_csv('../data/processed/605_processed.csv')
data_606 = pd.read_csv('../data/processed/606_processed.csv')

rawdata_605 = pd.read_csv('../data/rawdata_605.csv')
rawdata_606 = pd.read_csv('../data/rawdata_606.csv')

# Panel data
data_panel = pd.read_csv('../data/processed/regression_data_levels.csv')
data_panel.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
0,BNYC,2015 Q1,NASDAQ,11,213581,43704397,70249,43628268,0,43526470,...,0.115073,0.011198,-0.005493,0.105550,0.134962,Insigneo Securities,0.0,0.7766,0,NaN
1,BNYC,2015 Q1,NASDAQ,12,37042,7770176,4851,7750829,0,7713717,...,0.141117,0.005108,-0.004517,1.566054,0.528654,Insigneo Securities,0.0,0.6279,0,NaN
2,BNYC,2015 Q1,NYSE,11,396811,81015802,108105,80902077,0,80833088,...,0.070049,0.006949,-0.002302,0.094201,0.074817,Insigneo Securities,0.0,0.7379,0,NaN
3,BNYC,2015 Q1,NYSE,12,45238,9529134,5233,9506679,0,9478187,...,0.016105,0.003145,-0.002832,1.285204,-0.294374,Insigneo Securities,0.0,0.5228,0,NaN
4,BNYC,2015 Q1,Other,11,257920,55831472,108233,55715793,0,55640416,...,-0.023154,0.008315,-0.003278,0.331655,-0.098908,Insigneo Securities,0.0,0.7026,0,NaN


# Summary Stats

## Market Centers

In [6]:
data_605_dsc = pd.DataFrame()
data_605['MarketCenter'].unique()
data_605 = data_605.query('MarketCenter != "DBAB"')
data_605.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,PrImp_ExpAmt,All_AvgT,MktCtrAvg_PrImp_Pct,Rel_PrImp_Pct,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT
0,BNYC,2015Q1,NASDAQ,11,213581,43704397,70249,43628268,0,43526470,...,0.005705,0.240511,0.877085,0.061611,0.082561,0.115073,0.011198,-0.005493,0.105550,0.134962
1,BNYC,2015Q1,NASDAQ,12,37042,7770176,4851,7750829,0,7713717,...,0.000590,2.094708,0.661344,0.215484,0.188143,0.141117,0.005108,-0.004517,1.566054,0.528654
2,BNYC,2015Q1,NASDAQ,13,12616,2573500,501881,1976798,0,1048397,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
3,BNYC,2015Q1,NASDAQ,14,11996,2589515,641176,1838962,0,504154,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
4,BNYC,2015Q1,NASDAQ,15,36132,7574355,2619472,4075705,0,404016,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000


### Dates

In [7]:
data_605_dsc['First_Quarter_obs'] = data_605.groupby(['MarketCenter']).first()['Quarter']
data_605_dsc['Last_Quarter_obs']  = data_605.groupby(['MarketCenter']).last()['Quarter']
data_605_dsc = data_605_dsc.reset_index()
print(data_605_dsc.to_latex())

\begin{tabular}{llll}
\toprule
{} & MarketCenter & First\_Quarter\_obs & Last\_Quarter\_obs \\
\midrule
0 &         BNYC &            2015Q1 &           2017Q4 \\
1 &         CDRG &            2014Q4 &           2017Q4 \\
2 &         FBCO &            2010Q1 &           2018Q1 \\
3 &         G1ES &            2008Q2 &           2017Q4 \\
4 &         SGMA &            2010Q4 &           2017Q4 \\
5 &         UBSS &            2008Q2 &           2017Q4 \\
6 &         VRTU &            2017Q3 &           2018Q1 \\
7 &         WOLV &            2016Q4 &           2017Q4 \\
\bottomrule
\end{tabular}



### **Table 1:** Summary Statistics for Execution Quality Variables from 605 Data

In [8]:
descriptive_vars = ['PrImp_AvgAmt', 'PrImp_Pct', 'PrImp_ExpAmt', 'PrImp_AvgT', 'All_AvgT']

for exh in ['NASDAQ', 'NYSE', 'Other']:
    print(exh)
    print(data_605.query('OrderType == "Market" & Exchange == "' + exh + '"')[descriptive_vars].describe().transpose().drop(['count', 'std'], axis = 1).to_latex(float_format = '%0.5f'))
    
print(data_605.query('OrderType == "Market"')[descriptive_vars].describe().transpose().drop(['count', 'std'], axis = 1).to_latex(float_format = '%0.5f'))

NASDAQ
\begin{tabular}{lrrrrrr}
\toprule
{} &    mean &     min &     25\% &     50\% &     75\% &      max \\
\midrule
PrImp\_AvgAmt & 0.01186 & 0.00239 & 0.00610 & 0.01039 & 0.01679 &  0.02919 \\
PrImp\_Pct    & 0.84686 & 0.04850 & 0.84221 & 0.90108 & 0.93484 &  0.96306 \\
PrImp\_ExpAmt & 0.01022 & 0.00045 & 0.00495 & 0.00859 & 0.01503 &  0.02759 \\
PrImp\_AvgT   & 0.20220 & 0.00000 & 0.00855 & 0.03698 & 0.19712 &  8.23887 \\
All\_AvgT     & 0.34120 & 0.00000 & 0.02228 & 0.06334 & 0.27496 & 17.95130 \\
\bottomrule
\end{tabular}

NYSE
\begin{tabular}{lrrrrrr}
\toprule
{} &    mean &     min &     25\% &     50\% &     75\% &      max \\
\midrule
PrImp\_AvgAmt & 0.00672 & 0.00198 & 0.00420 & 0.00648 & 0.00893 &  0.01571 \\
PrImp\_Pct    & 0.86290 & 0.03728 & 0.85863 & 0.91970 & 0.94968 &  0.98118 \\
PrImp\_ExpAmt & 0.00589 & 0.00038 & 0.00349 & 0.00560 & 0.00822 &  0.01337 \\
PrImp\_AvgT   & 0.17096 & 0.00000 & 0.00549 & 0.02482 & 0.19239 &  5.88234 \\
All\_AvgT     & 0.25787 & 0.00119

### **Table A3:** Market Center Executions and Average Execution Quality

In [9]:
descriptive_vars = ['MktCtrExecShares', 'AwayExecShares', 'PrImp_AvgAmt', 'PrImp_Pct', 'PrImp_ExpAmt', 'PrImp_AvgT', 'All_AvgT']

temp = data_605.groupby(['MarketCenter', 'Exchange', 'OrderType']).mean()[descriptive_vars].reset_index()
data_605_dsc = data_605_dsc.merge(temp)
data_605_dsc.head()

,MarketCenter,First_Quarter_obs,Last_Quarter_obs,Exchange,OrderType,MktCtrExecShares,AwayExecShares,PrImp_AvgAmt,PrImp_Pct,PrImp_ExpAmt,PrImp_AvgT,All_AvgT
0,BNYC,2015Q1,2017Q4,NASDAQ,Limit,7.134647e+06,0.000000,0.002087,0.717611,0.001327,0.237141,1.782062
1,BNYC,2015Q1,2017Q4,NASDAQ,Market,3.897328e+07,16.666667,0.009661,0.890270,0.008555,0.196131,0.248774
2,BNYC,2015Q1,2017Q4,NASDAQ,Other,1.885684e+06,8.333333,NaN,0.000000,NaN,NaN,0.000000
3,BNYC,2015Q1,2017Q4,NYSE,Limit,9.242254e+06,0.000000,0.001271,0.726200,0.000835,0.179971,1.254090
4,BNYC,2015Q1,2017Q4,NYSE,Market,7.136117e+07,50.000000,0.006282,0.915504,0.005738,0.165398,0.192777


In [10]:
panels = ['NASDAQ', 'NYSE', 'Other']
data_output = data_605_dsc.copy()

data_output['MktCtrExecShares'] = data_output['MktCtrExecShares'].apply(lambda x: np.int(x))
data_output['AwayExecShares'] = data_output['AwayExecShares'].apply(lambda x: np.int(x))
data_output['PrImp_Pct'] = data_output['PrImp_Pct'].apply(lambda x: str(np.round(x*100, decimals = 2)) + '%')

for i in range(0, len(panels)):
   
    data_table = data_output.query('Exchange == "' + panels[i] + '"').query('OrderType == "Market"')
    data_table = data_table[['MarketCenter'] + descriptive_vars]
    raw_latex_table = data_table.to_latex(index = False)
    
    table_header = 'Panel ' + string.ascii_uppercase[i] + ': ' + panels[i] + ' Stocks'
    
    print(format_table_output(raw_latex_table, title = table_header), end = '\n\n')

\begin{table}
\small
\caption*{Panel A: NASDAQ Stocks}
\begin{tabular}{lrrrlrrr}
\toprule
MarketCenter &  MktCtrExecShares &  AwayExecShares &  PrImp\_AvgAmt & PrImp\_Pct &  PrImp\_ExpAmt &  PrImp\_AvgT &  All\_AvgT \\
\midrule
        BNYC &          38973278 &              16 &      0.009661 &    89.03\% &      0.008555 &    0.196131 &  0.248774 \\
        CDRG &         355233881 &           67733 &      0.020130 &    91.99\% &      0.018549 &    0.013842 &  0.036439 \\
        FBCO &           5356309 &          488076 &      0.009263 &    65.02\% &      0.005669 &    0.385073 &  0.626959 \\
        G1ES &         101387503 &          899320 &      0.010238 &    90.56\% &      0.009505 &    0.145026 &  0.170656 \\
        SGMA &          77452391 &               0 &      0.013399 &     90.0\% &      0.011977 &    0.007720 &  0.025985 \\
        UBSS &         142847729 &        24072755 &      0.011238 &    86.43\% &      0.009954 &    0.335845 &  0.673002 \\
        VRTU &        

## Brokers

In [11]:
# Find out which brokers accept rebates
broker_rebate_dict = {}

for broker in data_606['Broker'].unique():
    
    broker_rebate_dict[broker] = data_606[data_606['Broker'] == broker]['Rebate_Dummy'].iloc[0]
    
#brokers_paid = [brk for brk, rb in]

brokers_paid = [brk for brk in broker_rebate_dict if broker_rebate_dict[brk] == 1]
brokers_unpaid = [brk for brk in broker_rebate_dict if broker_rebate_dict[brk] == 0]

data_brokers_dsc = pd.DataFrame()

brokers = data_panel['Broker'].unique()

for broker in brokers:
    
    broker_rebate_dummy = data_panel.query('Broker == "' + broker + '"').iloc[0]['Rebate_Dummy']
    broker_connected_mktctrs = len(list(data_panel.query('Broker == "' + broker + '"')['MarketCenter'].unique()))
    
    data_brokers_dsc = data_brokers_dsc.append(
        {'Broker': broker, 'Rebate_Dummy': broker_rebate_dummy, 'MktCtr_Universe_Size': broker_connected_mktctrs}, 
        ignore_index=True)
    
    
data_brokers_dsc

,Broker,MktCtr_Universe_Size,Rebate_Dummy
0,Insigneo Securities,6.0,0.0
1,Bank of the West,6.0,0.0
2,Deutsche,6.0,1.0
3,Boenning Scattergood,7.0,1.0
4,Edward Jones,7.0,0.0
5,Credit Suisse,6.0,1.0
6,Hollencrest Securities,6.0,1.0
7,AXA,6.0,0.0
8,BTIG,4.0,1.0
9,Euro Pacific Capital,5.0,0.0


In [12]:
mktctrs = list(data_605['MarketCenter'].unique())

data_606_mktshr = data_panel.groupby(['MarketCenter', 'Broker']).mean().reset_index()

display(data_606_mktshr.query('Broker == "AXA"'))

temp = (1-data_606_mktshr.groupby(['Broker']).sum()).reset_index()
temp['MarketCenter'] = 'zzOther'
data_606_mktshr = data_606_mktshr.append(temp)

,MarketCenter,Broker,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,ExecShares_10_29,...,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Rebate,MktShare,Rebate_Dummy,Broker_Size
7,CDRG,AXA,11.5,7.691776e+06,1.196953e+09,6.790322e+08,5.177851e+08,6.871593e+04,5.176825e+08,85954.986111,...,0.247637,-0.123019,0.007392,0.000413,0.829694,-0.675203,0.0,0.182919,0.0,9999999.0
33,FBCO,AXA,11.5,6.789583e+04,1.383350e+07,4.810304e+04,1.322172e+07,5.630328e+05,1.377175e+07,6016.291667,...,0.201697,0.082578,0.006730,-0.002456,0.767821,0.240855,0.0,0.075254,0.0,9999999.0
48,G1ES,AXA,11.5,5.256432e+05,1.071749e+08,1.219182e+05,1.069851e+08,0.000000e+00,1.068746e+08,64811.197674,...,0.234701,-0.146926,0.007290,0.002928,0.835255,-0.348106,0.0,0.195986,0.0,9999999.0
59,SGMA,AXA,11.5,1.336170e+06,2.330731e+08,1.032962e+08,1.297738e+08,0.000000e+00,1.297377e+08,12136.117647,...,0.128498,-0.079786,0.007849,0.002856,0.730567,-0.562149,0.0,0.068274,0.0,9999999.0
75,UBSS,AXA,11.5,8.366569e+06,1.230690e+09,9.769899e+08,1.756340e+08,6.080746e+07,2.342838e+08,937121.031250,...,0.191693,0.083672,0.007342,0.000242,0.756657,0.974902,0.0,0.092428,0.0,9999999.0
101,VRTU,AXA,11.5,3.936978e+06,6.554066e+08,3.197718e+08,3.355861e+08,0.000000e+00,3.351193e+08,352245.500000,...,0.089636,0.013573,0.007866,0.000663,0.607954,-0.321408,0.0,0.115508,0.0,9999999.0


### **Table A2:** Broker Order Routing Averages

In [126]:
data_output = data_606_mktshr.copy()

temp = data_output[data_output['Broker'].apply(lambda x: x in brokers_paid)].groupby('MarketCenter').mean().reset_index().copy()
temp['Broker'] = 'zzPOF Average'
data_output = data_output.append(temp)

temp = data_output[data_output['Broker'].apply(lambda x: x in brokers_unpaid)].groupby('MarketCenter').mean().reset_index().copy()
temp['Broker'] = 'zzNon-POF Average'
data_output = data_output.append(temp)

temp = data_output.groupby('MarketCenter').mean().reset_index().copy()
temp['Broker'] = 'zzAll Average'
data_output = data_output.append(temp)

data_output['MktShare'] = data_output['MktShare'].fillna(0).apply(lambda x: np.round(x*100, decimals = 0))
data_output['MktShare'] = data_output['MktShare'].fillna(0).apply(lambda x: str(int(x)) + '%' if x > 0.00001 else '<1%')
data_output = data_output.pivot(columns = 'MarketCenter', values = 'MktShare', index = 'Broker')

#display(data_output)
#print(data_output.fillna('').to_latex().replace('<', '\\textless'))
print(data_output.loc[brokers_paid + ['zzPOF Average']].fillna('').to_latex().replace('<', '\\textless'))
print(data_output.loc[brokers_unpaid + ['zzNon-POF Average']].fillna('').to_latex().replace('<', '\\textless'))
print(data_output.loc[['zzAll Average']].fillna('').to_latex().replace('<', '\\textless'))

\begin{tabular}{llllllllll}
\toprule
MarketCenter & BNYC & CDRG & FBCO & G1ES & SGMA & UBSS & VRTU & WOLV & zzOther \\
Broker                 &      &      &      &      &      &      &      &      &         \\
\midrule
Deutsche               &   5\% &   1\% &   1\% &      &   1\% &   1\% &  \textless1\% &      &     91\% \\
Boenning Scattergood   &   2\% &  22\% &   4\% &   5\% &  10\% &  13\% &   8\% &      &     36\% \\
Evercore Group         &      &      &  11\% &      &      &      &      &      &     89\% \\
Credit Suisse          &  \textless1\% &  \textless1\% &  46\% &      &  \textless1\% &  \textless1\% &  \textless1\% &      &     53\% \\
Barclays Capital       &      &  \textless1\% &  \textless1\% &      &  \textless1\% &  \textless1\% &      &      &    100\% \\
Cambria Capital        &      &  10\% &      &      &      &  81\% &      &      &      9\% \\
JP Morgan              &      &  12\% &   5\% &      &      &  16\% &   4\% &      &     64\% \\
Inlet Securities   